In [8]:
from collections import defaultdict

In [10]:
grammar = {'A': ['SB', 'B'],'S': ['a', 'Bc', 'ε'],'B': ['b', 'd']}

In [12]:
def first(symbol):
    if symbol in first_set:
        return first_set[symbol]
    if not symbol.isupper():
        return {symbol}
    first_set[symbol] = set()
    for production in grammar[symbol]:
        has_epsilon = True
        for char in production:
            char_first = first(char)
            first_set[symbol].update(char_first - {'ε'})
            if 'ε' not in char_first:
                has_epsilon = False
                break
        if has_epsilon:
            first_set[symbol].add('ε')
    return first_set[symbol]

In [14]:
first_set = {}
for non_terminal in grammar:
    first(non_terminal)
    print(f"FIRST({non_terminal}) = {first_set[non_terminal]}")

FIRST(A) = {'a', 'b', 'd'}
FIRST(S) = {'ε', 'a', 'b', 'd'}
FIRST(B) = {'d', 'b'}


In [16]:
follow_set={}

In [18]:
def follow(symbol, start_symbol='A'):
    if symbol in follow_set:
        return follow_set[symbol]
    follow_set[symbol] = set()
    if symbol == start_symbol:  
        follow_set[symbol].add('$')
    for lhs, rhs_list in grammar.items():
        for rhs in rhs_list: 
            for i, sym in enumerate(rhs):
                if sym == symbol:  
                    if i + 1 < len(rhs): 
                        next_symbol = rhs[i + 1]
                        first_next = first(next_symbol) 
                        follow_set[symbol].update(first_next - {'ε'}) 
                        if 'ε' in first_next: 
                            follow_set[symbol].update(follow(lhs))
                    else:  
                        if lhs != symbol:  
                            follow_set[symbol].update(follow(lhs))
    return follow_set[symbol]

In [20]:
for non_terminal in grammar:
    follow(non_terminal)
print("FOLLOW sets:")
for nt, f_set in follow_set.items():
    print(f"FOLLOW({nt}) = {f_set}")

FOLLOW sets:
FOLLOW(A) = {'$'}
FOLLOW(S) = {'d', 'b'}
FOLLOW(B) = {'c', '$'}


In [39]:
parsing_table = defaultdict(dict)

In [49]:
for lhs, rhs_list in grammar.items():
    for rhs in rhs_list:
        first_rhs = set()
        has_epsilon = True
        for char in rhs:
            char_first = first(char)
            first_rhs.update(char_first - {'ε'})
            if 'ε' not in char_first:
                has_epsilon = False
                break
        if has_epsilon:
            first_rhs.add('ε')
        for terminal in first_rhs - {'ε'}:
            parsing_table[lhs][terminal] = rhs
        if 'ε' in first_rhs:
            for terminal in follow_set[lhs]:
                parsing_table[lhs][terminal] = rhs

In [74]:
parsing_table = defaultdict(lambda: defaultdict(str))

for lhs, rhs_list in grammar.items():
    for rhs in rhs_list:
        first_rhs = set()
        has_epsilon = True
        for char in rhs:
            char_first = first(char)
            first_rhs.update(char_first - {'ε'})
            if 'ε' not in char_first:
                has_epsilon = False
                break
        if has_epsilon:
            first_rhs.add('ε')
        for terminal in first_rhs - {'ε'}:
            parsing_table[lhs][terminal] = f"{lhs} → {rhs}"
        if 'ε' in first_rhs:
            for terminal in follow_set[lhs]:
                parsing_table[lhs][terminal] = f"{lhs} → {rhs}"

In [102]:
print("Parsing Table:")
terminals = sorted(set(t for row in parsing_table.values() for t in row.keys()))
print(f"{'NT':<5} | " + " | ".join(f"{t:<5}" for t in terminals))
print("-" * (7 + 7 * len(terminals)))

for nt in grammar.keys():
    row = [parsing_table[nt].get(t, "") for t in terminals]
    print(f"{nt:<5} | " + " | ".join(f"{entry:<5}" for entry in row))

Parsing Table:
NT    | a     | b     | d    
----------------------------
A     | A → SB | A → B | A → B
S     | S → a | S → ε | S → ε
B     |       | B → b | B → d
